In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import codecs
import os, glob
import time

In [2]:
def enlist_talk_names(path, dict_):
    r = urllib.request.urlopen(path)
    soup = BeautifulSoup(r.read(), "lxml")
    r.close()
    time.sleep(5)
    #print(soup)
    talks = soup.find_all("a", class_ = 'ga-link')
    for i in talks:
        if (i.attrs['href'].find('/talks/')==0 and dict_.get(i.attrs['href'])!=1):
            dict_[i.attrs['href']]=1
    return dict_

In [3]:
def get_paralle_source_sub(all_talk_names, source_paralle_sub):
    for i in all_talk_names:
        path = 'https://www.ted.com' + i[:-12]
        r = urllib.request.urlopen(path)
        soup = BeautifulSoup(r.read(), "lxml")
        r.close()
        time.sleep(5)
        condition = []
        print (path)
        for i in soup.find_all('link'):
            if(i.get('href') != None and i.get('hreflang') != None and i['href'].find('?language=') != -1):
                if(i['href'].find('?language=en') != -1 or i['href'].find('?language=vi') != -1):
                    condition.append(i['hreflang'])
                    print(i['hreflang'])
        if(len(condition) == 2):
            source_paralle_sub[path + '/transcript?language=en'] = path + '/transcript?language=vi'
    return source_paralle_sub

In [20]:
# Lấy subtitle
def get_sub(path):
    r = urllib.request.urlopen(path);
    soup = BeautifulSoup(r.read(), "lxml");
    r.close()
    time.sleep(5)
    span_include_sub = soup.find_all('div', {'class': ['Grid__cell', 'flx-s:1', 'p-r:4']})
    group = []
    sub_array_object = []
    for i in span_include_sub:
        if(i['class'] == ['Grid__cell', 'flx-s:1', 'p-r:4']):
            group.append(i.children)
    for i in group:
        next(i)
        sub = next(i)
        true_sub = sub.string
        sub_array_object.append(true_sub)
    return sub_array_object

In [5]:
# chi subtitle theo câu
def split_by_sentence(sub_array_object):
    text = ''
    final_sub_array =[]
    for j in sub_array_object:
        #dem = 0
        for i in j:
            if(i!='\n' and i!='\t'):
                text = text + i
            if(i =='.'):
                final_sub_array.append(text.strip())
                text = ''
            if(i == '\n'):
                i = ' '
                text = text + i
            #dem = dem + 1
    while ('' in final_sub_array):
        final_sub_array.remove('')
    while (' ' in final_sub_array):
        final_sub_array.remove(' ')
    return final_sub_array

In [6]:
# Chia subtitle theo đoạn
def split_by_paragraph(sub_array_object):
    text = ''
    final_sub_array =[]
    for i in sub_array_object:
        for j in range(len(i)):   
            if(i[j]!='\n' and i[j]!='\t'):
                text = text + i[j]
            if(i[j] == '\n'):
                text += ' '
            if(i[j] == '\t' and j + 1 > len(i)-1):
                final_sub_array.append(text.strip())
                text = ''
    while ('' in final_sub_array):
        final_sub_array.remove('')
    while (' ' in final_sub_array):
        final_sub_array.remove(' ')
    return final_sub_array

In [7]:
# length doc1 > length doc2
def flat_sentence(doc1, doc2):
    length1 = 0
    length2 = 0
    min = 0
    max = 0
    i = 0
    while(i < len(doc1) and i < len(doc2)):
        if (len(doc1) == len(doc2)):
            break
        doc1[i] = doc1[i].replace('  ',' ')
        doc2[i] = doc2[i].replace('  ',' ')
        length1 = len(doc1[i].split(' '))
        length2 = len(doc2[i].split(' '))
        #print(length1, ':', doc1[i])
        
        #print(length2, ':', doc2[i])
        min = length1
        if(length1 > length2):
            min = length2
        error = abs(length1-length2)/min
        #print(error)
        if (error > 1):
            #print(doc1[i])
            #print(doc2[i])
            #print('trung error:[[[[[[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]]]]]]')
            if(length1 < length2 and i < len(doc1)-1):
                doc1[i] = doc1[i] + doc1[i+1]
                del doc1[i+1]
                i+=1
            if(length1 > length2 and i < len(doc2)-1):
                doc2[i] = doc2[i] + doc2[i+1]
                del doc2[i+1]
                i+=1
        #print('=====================================================================')
        i+=1
    return doc1, doc2

In [8]:
# length doc1 > length doc2
def flat_paragraph(doc1, doc2):
    length1 = 0
    length2 = 0
    min = 0
    max = 0
    for i in range(len(doc1)):
        length1 = len(doc1[i].split(' '))
        length2 = len(doc2[i].split(' '))
        #print(length1)
        #print(length2)
        min = length1
        if(length1 > length2):
            min = length2
        error = abs(length1-length2)/min
        #print(error)
        if (error > 1):
            #print(doc1[i])
            #print(doc2[i])
            #print('trung error:[[[[[[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]]]]]]')
            if(length1 < length2 and '.' in doc2[i]):
                #print('vao day')
                split_index = doc2[i].index('.')
                split_data = doc2[i][split_index + 1:]
                doc2[i] = doc2[i][:split_index + 1]
                doc2.insert(i + 1, split_data)
            if(length1 > length2 and '.' in doc1[i]):
                split_index = doc1[i].index('.')
                split_data = doc1[i][split_index + 1:]
                doc1[i] = doc1[i][:split_index + 1]
                doc1.insert(i + 1, split_data)
        #print('=====================================================================')
    return doc1, doc2

In [33]:
def collection_sub(source_paralle_sub):
    
    for i in source_paralle_sub:
        en_sub_obj = get_sub(i)
        vi_sub_obj = get_sub(source_paralle_sub[i])
        
        print('Video title: ', source_paralle_sub[i][26:-23])
        print('split by paragraph')

        en_sub = split_by_paragraph(en_sub_obj)
        vi_sub = split_by_paragraph(vi_sub_obj)

        print('en: ', len(en_sub))
        print('vi: ', len(vi_sub))

        if(len(en_sub) != len(vi_sub)):
            max_len = len(en_sub)
            min_len = len(vi_sub)
            if(len(en_sub) < len(vi_sub)):
                max_len = len(vi_sub)
                min_len = len(en_sub)
            if(max_len/min_len > 2 and max_len - min_len > 5):
                en_sub = split_by_sentence(en_sub_obj)
                vi_sub = split_by_sentence(vi_sub_obj)
                print('fix -> split_by_sentence:')
                print('en: ', len(en_sub))
                print('vi: ', len(vi_sub))
                if(len(en_sub) != len(vi_sub) and abs(len(en_sub) - len(vi_sub)) < 3):
                    en_sub, vi_sub = flat_sentence(en_sub, vi_sub)
                    print('fix -> flat_by_sentence():')
                    print('en: ', len(en_sub))
                    print('vi: ', len(vi_sub))
                    if(len(en_sub) != len(vi_sub)):
                        print(len(en_sub) != len(vi_sub))
                        print('IGNORE DOCUMENT')
                        print('================================')
                        continue
                    print('================================')
                print('================================')
                continue
                
            print('fix -> flat_by_paragraph():')
            en_sub, vi_sub = flat_paragraph(en_sub, vi_sub)
            print('en: ', len(en_sub))
            print('vi: ', len(vi_sub))
            if(len(en_sub) != len(vi_sub)):
                en_sub = split_by_sentence(en_sub_obj)
                vi_sub = split_by_sentence(vi_sub_obj)
                print('fix -> split_by_sentence:')
                print('en: ', len(en_sub))
                print('vi: ', len(vi_sub))
                if(len(en_sub) != len(vi_sub) and abs(len(en_sub) - len(vi_sub)) < 3):
                    en_sub, vi_sub = flat_sentence(en_sub, vi_sub)
                    print('fix -> flat_by_sentence():')
                    print('en: ', len(en_sub))
                    print('vi: ', len(vi_sub))
                    if(len(en_sub) != len(vi_sub)):
                        print(len(en_sub) != len(vi_sub))
                        print('IGNORE DOCUMENT')
                        print('================================')
                        continue
        
        data_language = {
            'English' : pd.Series(en_sub),
            'Vietnamese' : pd.Series(vi_sub)
        }

        corpus_parallel_language = pd.DataFrame(data_language)
    
        corpus_parallel_language.to_excel(r'./collection/' + source_paralle_sub[i][26:-23] + '.xlsx', encoding='utf-8-sig')
        
        print('================================')
    

In [10]:
print('Get source video...')
all_talk_names = {}
for i in range(6,7):
    PATH = 'https://www.ted.com/talks?language=vi&page=%d&sort=newest'%i
    all_talk_names = enlist_talk_names(PATH, all_talk_names)
all_talk_names

Get source video...


{'/talks/alex_gendler_epic_engineering_building_the_brooklyn_bridge?language=vi': 1,
 '/talks/robert_reffkin_5_ways_to_create_stronger_connections?language=vi': 1,
 '/talks/leeann_renninger_the_secret_to_giving_great_feedback?language=vi': 1,
 '/talks/rahaf_harfoush_how_burnout_makes_us_less_creative?language=vi': 1,
 '/talks/patrick_mcginnis_how_to_make_faster_decisions?language=vi': 1,
 '/talks/alex_gendler_everything_changed_when_the_fire_crystal_got_stolen?language=vi': 1,
 '/talks/smruti_jukur_johari_what_if_the_poor_were_part_of_city_planning?language=vi': 1,
 '/talks/ellen_agler_parasitic_worms_hold_back_human_progress_here_s_how_we_can_end_them?language=vi': 1,
 '/talks/sylvain_duranton_how_humans_and_ai_can_work_together_to_create_better_businesses?language=vi': 1,
 '/talks/jessica_ochoa_hendrix_how_virtual_reality_turns_students_into_scientists?language=vi': 1,
 '/talks/christopher_bahl_a_new_type_of_medicine_custom_made_with_tiny_proteins?language=vi': 1,
 '/talks/melody_smi

In [11]:
print('Get source subtitle...')
source_paralle_sub = {}
source_paralle_sub = get_paralle_source_sub(all_talk_names, source_paralle_sub)

Get source subtitle...
https://www.ted.com/talks/alex_gendler_epic_engineering_building_the_brooklyn_bridge
en
vi
https://www.ted.com/talks/robert_reffkin_5_ways_to_create_stronger_connections
en
vi
https://www.ted.com/talks/leeann_renninger_the_secret_to_giving_great_feedback
en
vi
https://www.ted.com/talks/rahaf_harfoush_how_burnout_makes_us_less_creative
en
vi
https://www.ted.com/talks/patrick_mcginnis_how_to_make_faster_decisions
en
vi
https://www.ted.com/talks/alex_gendler_everything_changed_when_the_fire_crystal_got_stolen
en
vi
https://www.ted.com/talks/smruti_jukur_johari_what_if_the_poor_were_part_of_city_planning
en
vi
https://www.ted.com/talks/ellen_agler_parasitic_worms_hold_back_human_progress_here_s_how_we_can_end_them
en
vi
https://www.ted.com/talks/sylvain_duranton_how_humans_and_ai_can_work_together_to_create_better_businesses
en
vi
https://www.ted.com/talks/jessica_ochoa_hendrix_how_virtual_reality_turns_students_into_scientists
en
vi
https://www.ted.com/talks/christo

In [34]:
collection_sub(source_paralle_sub)

Video title:  alex_gendler_epic_engineering_building_the_brooklyn_bridge
split by paragraph
en:  8
vi:  8
Video title:  robert_reffkin_5_ways_to_create_stronger_connections
split by paragraph
en:  10
vi:  10
Video title:  leeann_renninger_the_secret_to_giving_great_feedback
split by paragraph
en:  13
vi:  13
Video title:  rahaf_harfoush_how_burnout_makes_us_less_creative
split by paragraph
en:  13
vi:  13
Video title:  patrick_mcginnis_how_to_make_faster_decisions
split by paragraph
en:  20
vi:  20
Video title:  alex_gendler_everything_changed_when_the_fire_crystal_got_stolen
split by paragraph
en:  14
vi:  14
Video title:  smruti_jukur_johari_what_if_the_poor_were_part_of_city_planning
split by paragraph
en:  25
vi:  25
Video title:  ellen_agler_parasitic_worms_hold_back_human_progress_here_s_how_we_can_end_them
split by paragraph
en:  16
vi:  17
fix -> flat_by_paragraph():
en:  16
vi:  17
fix -> split_by_sentence:
en:  51
vi:  46
Video title:  sylvain_duranton_how_humans_and_ai_can_w

In [32]:
print(len(all_talk_names)) 

36
